# 모바일 적용을 위한 모델 경량화

In [1]:
import torch

from classifier.models.cnn1d import PositionClassifier

input_size = 4
output_size = 240
pattern_n = 5

c:\Users\seung\anaconda3\envs\DL\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
weights = "./weights/model.pt"
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def load(fn, device):
    d = torch.load(fn, map_location=device)
    
    return d['model'], d['config']

model_dict, train_config = load(weights, device)
print(train_config)

Namespace(weights='./weights/model.pt', gpu_id=-1, train_ratio=0.8, n_epochs=100, batch_size=256, dropout_p=0.3, verbose=1)


In [3]:
model = PositionClassifier(input_size, output_size).to(device)
model.eval()
model.load_state_dict(model_dict)

<All keys matched successfully>

In [4]:
from torch.utils.mobile_optimizer import optimize_for_mobile

example = torch.rand(pattern_n,input_size).unsqueeze(0)
print(example)
traced_script_module = torch.jit.trace(model, example)
optimized_traced_model = optimize_for_mobile(traced_script_module)
optimized_traced_model.save("./weights/for_mobile.pt")

tensor([[[0.6015, 0.7037, 0.6672, 0.3546],
         [0.8092, 0.5324, 0.3048, 0.1059],
         [0.3752, 0.3426, 0.0354, 0.4394],
         [0.6023, 0.1607, 0.5960, 0.6148],
         [0.5502, 0.0497, 0.7952, 0.9177]]])
